In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shutil
import random

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import shutil
import random

# Set paths for the original dataset and new directories
base_dir = '/content/drive/MyDrive/Dataset'
train_dir = '/content/drive/MyDrive/Dataset/train'
val_dir = '/content/drive/MyDrive/Dataset/val'

# Create directories for train and val splits
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(os.path.join(train_dir, 'eczema'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'normal'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'eczema'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'normal'), exist_ok=True)

# Function to split images into train/val
def split_data(source_dir, train_dir, val_dir, val_size=0.2):
    for category in ['eczema', 'normal']:
        category_dir = os.path.join(source_dir, category)
        files = os.listdir(category_dir)
        random.shuffle(files)
        val_count = int(len(files) * val_size)

        # Move files to train and val directories
        for file in files[val_count:]:
            shutil.move(os.path.join(category_dir, file), os.path.join(train_dir, category, file))
        for file in files[:val_count]:
            shutil.move(os.path.join(category_dir, file), os.path.join(val_dir, category, file))

# Split the data into train and val
split_data(base_dir, train_dir, val_dir)


In [5]:
# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(150),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(40),
    transforms.RandomAffine(0, shear=0.2, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

val_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load datasets
train_dataset = ImageFolder(train_dir, transform=train_transform)
val_dataset = ImageFolder(val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load pre-trained MobileNetV2 and modify it
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)  # Binary classification


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 99.9MB/s]


In [6]:
# Freeze all layers except the final classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            preds = torch.sigmoid(outputs) > 0.5
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.float().to(device)

                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)

                preds = torch.sigmoid(outputs) > 0.5
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

            val_loss = val_loss / len(val_dataset)
            val_acc = val_corrects.double() / len(val_dataset)

            print(f'Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}')


In [8]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Save the model
model_save_path = '/content/drive/MyDrive/eczema_model.pth'  # Save in your Google Drive
torch.save(model.state_dict(), model_save_path)

# Load the model for inference
model_load_path = '/content/drive/MyDrive/eczema_model.pth'  # Load from your Google Drive
model.load_state_dict(torch.load(model_load_path))

# Function to classify images in a directory
def classify_images_in_directory(directory_path, model, device):
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                img_path = os.path.join(root, file)
                img = Image.open(img_path).convert('RGB')
                img_tensor = transform(img).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = model(img_tensor).squeeze()
                    prediction = torch.sigmoid(output).item()

                if prediction < 0.5:
                    print(f"{file}: Predicted class = Eczema")
                else:
                    print(f"{file}: Predicted class = Normal")

# Test on unseen images
unseen_images_dir = '/unseen.jpg'
classify_images_in_directory(unseen_images_dir, model, device)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Loss: 0.3688, Acc: 0.8549
Validation Loss: 0.3720, Validation Acc: 0.8611
Epoch 2/10, Loss: 0.3659, Acc: 0.8571
Validation Loss: 0.3667, Validation Acc: 0.8611
Epoch 3/10, Loss: 0.3621, Acc: 0.8549
Validation Loss: 0.3651, Validation Acc: 0.8611
Epoch 4/10, Loss: 0.3546, Acc: 0.8549
Validation Loss: 0.3615, Validation Acc: 0.8611
Epoch 5/10, Loss: 0.3368, Acc: 0.8549
Validation Loss: 0.3576, Validation Acc: 0.8611
Epoch 6/10, Loss: 0.3348, Acc: 0.8549
Validation Loss: 0.3540, Validation Acc: 0.8611
Epoch 7/10, Loss: 0.3368, Acc: 0.8549
Validation Loss: 0.3479, Validation Acc: 0.8611
Epoch 8/10, Loss: 0.3261, Acc: 0.8594
Validation Loss: 0.3498, Validation Acc: 0.8611
Epoch 9/10, Loss: 0.3218, Acc: 0.8639
Validation Loss: 0.3421, Validation Acc: 0.8611
Epoch 10/10, Loss: 0.3048, Acc: 0.8617
Validation Loss: 0.3386, Validation Acc: 0.8611


<ipython-input-8-f14bcf35c274>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_load_path))
